<a href="https://colab.research.google.com/github/YoonieJang/CS484/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import math
import sklearn.metrics as metrics
from itertools import combinations
from google.colab import files
import io
pd.options.mode.chained_assignment = None  # default='warn'


### Question 2

In [ ]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['claim_history.csv']))

Saving claim_history.csv to claim_history.csv


In [ ]:
#Feature selection
features=df[["CAR_TYPE","OCCUPATION","EDUCATION"]]

train_data, test_data = train_test_split(df,test_size = 0.3, random_state=60616, stratify = df["CAR_USE"])

In [ ]:
#Ordinal order
train_data['EDUCATION'] = train_data['EDUCATION'].map(
    {'Below High School': 0, 'High School': 1, 'Bachelors': 2, 'Masters': 3, 'Doctors': 4})
test_data['EDUCATION'] = test_data['EDUCATION'].map(
    {'Below High School': 0, 'High School': 1, 'Bachelors': 2, 'Masters': 3, 'Doctors': 4})

In [ ]:
#a)What is the entropy value of the root node?
count = 0
for i in df["CAR_USE"]:
    if i == "Commercial":
       count +=1
proba_commercial = count/len(df["CAR_USE"])

proba_private = (len(df["CAR_USE"])-count)/len(df["CAR_USE"])

root_entropy = -((proba_commercial * np.log2(proba_commercial) + proba_private * np.log2(proba_private)))
print("The entropy value of the root node is",root_entropy)

The entropy value of the root node is 0.9489621493401781


In [ ]:
def split(data, split = 1, typ = 'EO', subset = None): 
    # splits according to the parameter
    if typ == 'EO':
        data['LE_Split'] = (data.iloc[:,0] <= split)
    elif typ == 'EN':
        data['LE_Split'] = data.iloc[:, 0].apply(lambda x: True if x in subset else False)
    crossTable = pd.crosstab(index=data['LE_Split'], 
                              columns=data.iloc[:, 1], margins=True, 
                              dropna=True)
   #print(crossTable)
    nRows = crossTable.shape[0]
    nColumns = crossTable.shape[1]
   
    tableEntropy = 0
    for iRow in range(nRows-1):
      rowEntropy = 0
      for iColumn in range(nColumns):
          proportion = crossTable.iloc[iRow,iColumn] / crossTable.iloc[iRow,(nColumns-1)]
          if (proportion > 0):
            rowEntropy -= proportion * np.log2(proportion)
      tableEntropy += rowEntropy *  crossTable.iloc[iRow,(nColumns-1)]
    tableEntropy = tableEntropy /  crossTable.iloc[(nRows-1),(nColumns-1)]
  
    return crossTable, tableEntropy


In [ ]:
def MinOrdinalEntropy(data, intervals):
    min_entropy = 999999999.99999999999
    min_interval = None
    min_table = None
    # tries out all splits possible and finds the min
    for i in range(intervals[0], intervals[len(intervals) - 1]):
        cur_table, cur_entropy = split(data, i + 0.5, typ = 'EO')
        if cur_entropy < min_entropy:
            min_entropy = cur_entropy
            min_interval = i + 0.5
            min_table = cur_table
    return min_entropy, min_interval

def MinNominalEntropy(data, sett):
  
    subset_map = {}
    # tries out all combinations
    for i in range(1, (int(len(sett) / 2)) + 1):
        subsets = combinations(sett, i)
        for ss in subsets:
            remaining = tuple()
            for ele in sett:
                if ele not in ss:
                    remaining += (ele,)
            if subset_map.get(remaining) == None:
                subset_map[ss] = remaining
    min_entropy = 9999999.9999999
    min_subset1 = min_subset2 = min_table = None
    for subset_total in subset_map:
        table, entropy = split(data, typ = 'EN', subset = subset_total)
        if entropy < min_entropy:
            min_entropy = entropy
            min_subset1 = subset_total
            min_subset2 = subset_map.get(subset_total)
            min_table = table
    return min_entropy, min_subset1, min_subset2

In [ ]:
def calculate_min_entropy(df,variable,combinations):
    inData1 = df[[variable,"CAR_USE"]]
    entropies = []
    for i in combinations:
        EV = split(inData1, list(i),typ = 'EO')
        entropies.append((EV,i))
    return min(entropies)

In [ ]:
#b)List the optimal split (i.e., feature name, values in the two branches, and the split entropy ) for all three features in the first layer.
entropy_education = MinOrdinalEntropy(train_data[['EDUCATION','CAR_USE']],[0,1,2,3,4])
entropy_education

(0.9353838400305854, 0.5)

In [ ]:
entropy_car_type = MinNominalEntropy(train_data[['CAR_TYPE', 'CAR_USE']],['Minivan', 'Panel Truck', 'Pickup', 'SUV', 
                                            'Sports Car', 'Van'])
entropy_car_type

(0.7666250073253262,
 ('Minivan', 'SUV', 'Sports Car'),
 ('Panel Truck', 'Pickup', 'Van'))

In [ ]:
entropy_occupation = MinNominalEntropy(train_data[['OCCUPATION','CAR_USE']],
                                 ['Blue Collar', 'Clerical', 'Doctor', 'Home Maker','Lawyer', 'Manager', 'Professional', 'Student', 'Unknown'])
entropy_occupation

(0.7193463163551073,
 ('Blue Collar', 'Student', 'Unknown'),
 ('Clerical', 'Doctor', 'Home Maker', 'Lawyer', 'Manager', 'Professional'))

In [ ]:
print("Car type entropy:",entropy_car_type, 
      "\nOccupation entropy:",entropy_occupation,
      "\nEducation entropy:",entropy_education)

Car type entropy: (0.7666250073253262, ('Minivan', 'SUV', 'Sports Car'), ('Panel Truck', 'Pickup', 'Van')) 
Occupation entropy: (0.7193463163551073, ('Blue Collar', 'Student', 'Unknown'), ('Clerical', 'Doctor', 'Home Maker', 'Lawyer', 'Manager', 'Professional')) 
Education entropy: (0.9353838400305854, 0.5)


##### c) Which feature is selected for splitting in the first layer?  
When I compare three features in the first layer, the entropy splitted by occupation is the lowest. The optimal split for all three features in the first layer would be Occupation with 2 branches: ['Blue Collar', 'Student', 'Unknown'] and ['Clerical', 'Doctor', 'Home Maker', 'Lawyer', 'Manager', 'Professional']

##### d) Which features are selected for splitting in the second layer?  What are the values in the branches of the second layer? 


In [ ]:
train_layer1_left = train_data[(train_data["OCCUPATION"] == "Blue Collar") | 
                               (train_data["OCCUPATION"] == "Unknown") | 
                               (train_data["OCCUPATION"] == "Student")]
train_layer1_right =  train_data[(train_data["OCCUPATION"] != "Blue Collar") & 
                                 (train_data["OCCUPATION"] != "Unknown") & 
                                 (train_data["OCCUPATION"] != "Student")]

In [ ]:
left_layer1_car_type = MinNominalEntropy(train_layer1_left[['CAR_TYPE', 'CAR_USE']],
                                  ['Minivan', 'Panel Truck', 'Pickup', 'SUV','Sports Car', 'Van'])
left_layer1_car_type

(0.7751126066962271,
 ('Minivan', 'SUV', 'Sports Car'),
 ('Panel Truck', 'Pickup', 'Van'))

In [ ]:
left_layer1_occupation = MinNominalEntropy(train_layer1_left[['OCCUPATION', 'CAR_USE']],
                                  ['Blue Collar','Student','Unknown'])
left_layer1_occupation

(0.8054718032426555, ('Student',), ('Blue Collar', 'Unknown'))

In [ ]:
left_layer1_education = MinOrdinalEntropy(train_layer1_left[['EDUCATION','CAR_USE']],[0,1,2,3,4])
left_layer1_education

(0.6789590813062101, 0.5)

In [ ]:
right_layer1_car_type = MinNominalEntropy(train_layer1_right[['CAR_TYPE', 'CAR_USE']],
                                  ['Minivan', 'Panel Truck', 'Pickup', 'SUV','Sports Car', 'Van'])
right_layer1_car_type

(0.3358099954089214,
 ('Minivan', 'SUV', 'Sports Car'),
 ('Panel Truck', 'Pickup', 'Van'))

In [ ]:
right_layer1_occupation = MinNominalEntropy(train_layer1_right[['OCCUPATION', 'CAR_USE']],
                                  ['Clerical', 'Doctor', 'Home Maker','Lawyer', 'Manager', 'Professional'])
right_layer1_occupation

(0.5757772820977519,
 ('Clerical', 'Manager', 'Professional'),
 ('Doctor', 'Home Maker', 'Lawyer'))

In [ ]:
right_layer1_education = MinOrdinalEntropy(train_layer1_right[['EDUCATION','CAR_USE']],[0,1,2,3,4])
right_layer1_education

(0.6267253367279664, 2.5)

In [ ]:
print('Left Splitted')
print("Car type entropy:",left_layer1_car_type[0], 
      "\nOccupation entropy:",left_layer1_occupation[0],
      "\nEducation entropy:",left_layer1_education[0])
print('\nRight Splitted')
print("Car type entropy:",right_layer1_car_type[0], 
      "\nOccupation entropy:",right_layer1_occupation[0],
      "\nEducation entropy:",right_layer1_education[0])

Left Splitted
Car type entropy: 0.7751126066962271 
Occupation entropy: 0.8054718032426555 
Education entropy: 0.6789590813062101

Right Splitted
Car type entropy: 0.3358099954089214 
Occupation entropy: 0.5757772820977519 
Education entropy: 0.6267253367279664


In [ ]:
print('Education will be selected on the first layer of left split with 2 branches:(Below High School), (High School, Bachelors,Masters,Doctors)')
print('Car type will be selected on the first layer of right split with 2 branches:',right_layer1_car_type[1],right_layer1_car_type[2])


Education will be selected on the first layer of left split with 2 branches:(Below High School), (High School, Bachelors,Masters,Doctors)
Car type will be selected on the first layer of right split with 2 branches: ('Minivan', 'SUV', 'Sports Car') ('Panel Truck', 'Pickup', 'Van')


In [ ]:
leave1 = train_layer1_left[train_layer1_left['EDUCATION'] <= right_layer1_education[0]]
print("Leave 1 (left to left)")
print("Decision rule: ", "Occupation with", entropy_occupation[1], '-> EDUCATION Level: Below High School')
print("Counts of the target categories:", leave1.shape[0])
print("Commercial:",leave1.groupby('CAR_USE').size()['Commercial'])
print("Private:",leave1.groupby('CAR_USE').size()['Private'])
print("Predicted probabilities for CAR_USE for commercial:",leave1.groupby('CAR_USE').size()['Commercial']/leave1.shape[0])

Leave 1 (left to left)
Decision rule:  Occupation with ('Blue Collar', 'Student', 'Unknown') -> EDUCATION Level: Below High School
Counts of the target categories: 578
Commercial: 155
Private: 423
Predicted probabilities for CAR_USE for commercial: 0.2681660899653979


In [ ]:
leave2 = train_layer1_left[train_layer1_left['EDUCATION'] > right_layer1_education[0]]
print("Leave 2 (left to right)")
print("Decision rule: ", "Occupation with", entropy_occupation[1], '-> EDUCATION Level: High School, Bachelors,Masters,Doctors')
print("Counts of the target categories:", leave2.shape[0])
print("Commercial:",leave2.groupby('CAR_USE').size()['Commercial'])
print("Private:",leave2.groupby('CAR_USE').size()['Private'])
print("Predicted probabilities for CAR_USE for commercial:",leave2.groupby('CAR_USE').size()['Commercial']/leave2.shape[0])

Leave 2 (left to right)
Decision rule:  Occupation with ('Blue Collar', 'Student', 'Unknown') -> EDUCATION Level: High School, Bachelors,Masters,Doctors
Counts of the target categories: 2108
Commercial: 1770
Private: 338
Predicted probabilities for CAR_USE for commercial: 0.8396584440227703


In [ ]:
leave3 = train_layer1_right[train_layer1_right['CAR_TYPE'].isin(right_layer1_car_type[1])]
print("Leave 3 (right to left)")
print("Decision rule: ", "Occupation with", entropy_occupation[2], '-> Car type:',right_layer1_car_type[1])
print("Counts of the target categories:", leave3.shape[0])
print("Commercial:",leave3.groupby('CAR_USE').size()['Commercial'])
print("Private:",leave3.groupby('CAR_USE').size()['Private'])
print("Predicted probabilities for CAR_USE for commercial:",leave3.groupby('CAR_USE').size()['Commercial']/leave3.shape[0])

Leave 3 (right to left)
Decision rule:  Occupation with ('Clerical', 'Doctor', 'Home Maker', 'Lawyer', 'Manager', 'Professional') -> Car type: ('Minivan', 'SUV', 'Sports Car')
Counts of the target categories: 3225
Commercial: 27
Private: 3198
Predicted probabilities for CAR_USE for commercial: 0.008372093023255815


In [ ]:
leave4 = train_layer1_right[train_layer1_right['CAR_TYPE'].isin(right_layer1_car_type[2])]
print("Leave 4 (right to right)")
print("Decision rule: ", "Occupation with", entropy_occupation[2], '-> Car type:',right_layer1_car_type[2])
print("Counts of the target categories:", leave4.shape[0])
print("Commercial:",leave4.groupby('CAR_USE').size()['Commercial'])
print("Private:",leave4.groupby('CAR_USE').size()['Private'])
print("Predicted probabilities for CAR_USE for commercial:",leave4.groupby('CAR_USE').size()['Commercial']/leave4.shape[0])

Leave 4 (right to right)
Decision rule:  Occupation with ('Clerical', 'Doctor', 'Home Maker', 'Lawyer', 'Manager', 'Professional') -> Car type: ('Panel Truck', 'Pickup', 'Van')
Counts of the target categories: 1300
Commercial: 700
Private: 600
Predicted probabilities for CAR_USE for commercial: 0.5384615384615384


### Question 3

In [ ]:
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import scipy
import statsmodels.api as stats

In [ ]:
import io
uploaded = files.upload()
df2 = pd.read_csv(io.BytesIO(uploaded['sample_v10.csv']))
trainData = df2.dropna()

Saving sample_v10.csv to sample_v10 (1).csv


In [ ]:
#a) frequency table
FreqTable = pd.crosstab(index=trainData['y'],columns=["Frequency"],margins = True, dropna = True)
FreqTable= FreqTable.drop(columns='All')
print("Frequency Table")
print(FreqTable)

Frequency Table
col_0  Frequency
y               
1           2274
2           3532
3           4194
All        10000


In [ ]:
# Rename the columns
# Specify y as a categorical variable
Origin = df2['y'].astype('category')
y = Origin
y_category = y.cat.categories
catFeature = ['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']

In [ ]:
#b)
# Backward Selection
# Consider Model 0 is y = intercept + x1 + x2+ x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10
DriveTrain = df2[['x1']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x2','x3','x4','x5','x6','x7','x8','x9','x10']])
X = stats.add_constant(X, prepend=True)
DF1 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK1 = logit.loglike(thisParameter.values)
NSample = len(y)
print(thisFit.summary())
print("\nInitial model: y = intercept + x1+x2+x3+x4+x5+x6+x7+x8+x9+x10 ")
print("Model Log-Likelihood Value =", LLK1)
print("Number of Free Parameters =", DF1)

AIC = 2.0*DF1 - 2.0*LLK1
BIC = DF1 *np.log(NSample) - 2.0 * LLK1
print("\n\nAIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.195606
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9978
Method:                           MLE   Df Model:                           20
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.8170
Time:                        01:44:16   Log-Likelihood:                -1956.1
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0165      0.087     11.636      0.000       0.845       1.188
x1            -1.1172      0

In [ ]:
# Consider Model is y = intercept + x2+ x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10
DriveTrain = df2[['x2']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x3','x4','x5','x6','x7','x8','x9','x10']])
X = stats.add_constant(X, prepend=True)
DF0 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK0 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK0)
DF = DF1 - DF0
pValue = scipy.stats.chi2.sf(Deviance, DF)

print(thisFit.summary())
print("Removed feature: x1")
print("Model Log-Likelihood Value =", LLK0)
print("Number of Free Parameters =", DF0)
print("Deviance Test Statistic: ", Deviance,"\nDeviance Degree of Freedom: ",DF,"\nDeviance Significance Value: ", pValue)

AIC = 2.0*DF0 - 2.0*LLK0
BIC = DF0 *np.log(NSample) - 2.0 * LLK0
print("\nAIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.222527
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9980
Method:                           MLE   Df Model:                           18
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.7918
Time:                        01:44:24   Log-Likelihood:                -2225.3
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7927      0.079     10.042      0.000       0.638       0.947
x2            -0.0118      0

In [ ]:
# Consider Model 1 is y = intercept + x1 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10
DriveTrain = df2[['x3']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x1','x4','x5','x6','x7','x8','x9','x10']])
X = stats.add_constant(X, prepend=True)
DF2 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK2 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK2)
DF = DF1 - DF0
pValue = scipy.stats.chi2.sf(Deviance, DF)

print(thisFit.summary())
print("Removed feature: x2")
print("Model Log-Likelihood Value =", LLK2)
print("Number of Free Parameters =", DF2)
print("Deviance (Test Statistic: ", Deviance, ") Deviance (Degree of Freedom: ",DF,") Deviance (Significance Value: ", pValue,")")

AIC = 2.0*DF2 - 2.0*LLK2
BIC = DF2 *np.log(NSample) - 2.0 * LLK2
print("\nAIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.195633
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9980
Method:                           MLE   Df Model:                           18
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.8170
Time:                        01:44:39   Log-Likelihood:                -1956.3
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0166      0.087     11.637      0.000       0.845       1.188
x3             0.0105      0

In [ ]:
# Consider Model 1 is y = intercept + x1 + x2 + x4 + x5 + x6 + x7 + x8 + x9 + x10
DriveTrain = df2[['x4']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x1','x2','x5','x6','x7','x8','x9','x10']])
X = stats.add_constant(X, prepend=True)
DF3 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK3 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK3)
DF = DF1 - DF3
pValue = scipy.stats.chi2.sf(Deviance, DF)

print(thisFit.summary())
print("Removed feature: x3")
print("Model Log-Likelihood Value =", LLK3)
print("Number of Free Parameters =", DF3)
print("Deviance (Test Statistic: ", Deviance, ") Deviance (Degree of Freedom: ",DF,") Deviance (Significance Value: ", pValue,")")

AIC = 2.0*DF3 - 2.0*LLK3
BIC = DF3 *np.log(NSample) - 2.0 * LLK3
print("\nAIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.195628
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9980
Method:                           MLE   Df Model:                           18
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.8170
Time:                        01:44:56   Log-Likelihood:                -1956.3
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0166      0.087     11.637      0.000       0.845       1.188
x4            -1.5570      0

In [ ]:
# Consider Model 1 is y = intercept + x1 + x2+ x3 + x5 + x6 + x7 + x8 + x9 + x10
DriveTrain = df2[['x5']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x1','x2','x3','x6','x7','x8','x9','x10']])
X = stats.add_constant(X, prepend=True)
DF4 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK4 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK4)
DF = DF1 - DF4
pValue = scipy.stats.chi2.sf(Deviance, DF)
print(thisFit.summary())
print("Removed feature: x4")
print("Model Log-Likelihood Value =", LLK4)
print("Number of Free Parameters =", DF4)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)
AIC = 2.0*DF4 - 2.0*LLK4
BIC = DF4 *np.log(NSample) - 2.0 * LLK4
print("\nAIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.578049
         Iterations 9
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9980
Method:                           MLE   Df Model:                           18
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.4592
Time:                        01:46:23   Log-Likelihood:                -5780.5
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8534      0.040     21.090      0.000       0.774       0.933
x5             0.0028      0.

In [ ]:
# Consider Model 1 is y = intercept + x1 + x2+ x3 + x4 + x6 + x7 + x8 + x9 + x10
DriveTrain = df2[['x6']]
X =pd.get_dummies(DriveTrain)
X = X.join(df2[['x1','x2','x3','x4','x7','x8','x9','x10']])
X = stats.add_constant(X, prepend=True)
DF5 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK5 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK5)
DF = DF1 - DF5
pValue = scipy.stats.chi2.sf(Deviance, DF)
print(thisFit.summary())
print("Removed feature: x5")
print("Model Log-Likelihood Value =", LLK5)
print("Number of Free Parameters =", DF5)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)

AIC = 2.0*DF5 - 2.0*LLK5
BIC = DF5 *np.log(NSample) - 2.0 * LLK5
print("\nAIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.195647
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9980
Method:                           MLE   Df Model:                           18
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.8169
Time:                        01:47:22   Log-Likelihood:                -1956.5
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0167      0.087     11.644      0.000       0.846       1.188
x6             0.0163      0

In [ ]:
# Consider Model 1 is y = intercept + x1 + x2+ x3 + x4 + x5 + x7 + x8 + x9 + x10
DriveTrain = df2[['x7']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x1','x2','x3','x4','x5','x8','x9','x10']])
X = stats.add_constant(X, prepend=True)
DF6 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK6 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK6)
DF = DF1 - DF6
pValue = scipy.stats.chi2.sf(Deviance, DF)
print(thisFit.summary())
print("Removed value: x6")
print("Model Log-Likelihood Value =", LLK6)
print("Number of Free Parameters =", DF6)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)

AIC = 2.0*DF6 - 2.0*LLK6
BIC = DF6 *np.log(NSample) - 2.0 * LLK6
print("\nAIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.195808
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9980
Method:                           MLE   Df Model:                           18
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.8168
Time:                        01:48:08   Log-Likelihood:                -1958.1
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0175      0.087     11.654      0.000       0.846       1.189
x7            -0.0006      0

In [ ]:
# Consider Model 1 is y = intercept + x1 + x2+ x3 + x4 + x5 + x6 + x8 + x9 + x10
DriveTrain = df2[['x8']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x1','x2','x3','x4','x5','x6','x9','x10']])
X = stats.add_constant(X, prepend=True)
DF7 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK7 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK7)
DF = DF1 - DF7
pValue = scipy.stats.chi2.sf(Deviance, DF)
print(thisFit.summary())
print("Removed value: x7")
print("Model Log-Likelihood Value =", LLK7)
print("Number of Free Parameters =", DF7)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)

AIC = 2.0*DF7 - 2.0*LLK7
BIC = DF7 *np.log(NSample) - 2.0 * LLK7
print("\nAIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.195607
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9980
Method:                           MLE   Df Model:                           18
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.8170
Time:                        01:49:11   Log-Likelihood:                -1956.1
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0165      0.087     11.636      0.000       0.845       1.188
x8            -0.0134      0

In [ ]:
# Consider Model 1 is y = intercept + x1 + x2+ x3 + x4 + x5 + x6 + x7 + x9 + x10
DriveTrain = df2[['x9']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x1','x2','x3','x4','x5','x6','x7','x10']])
X = stats.add_constant(X, prepend=True)
DF8 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK8 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK8)
DF = DF1 - DF8
pValue = scipy.stats.chi2.sf(Deviance, DF)
print(thisFit.summary())
print("Removed value: x8")
print("Model Log-Likelihood Value =", LLK8)
print("Number of Free Parameters =", DF8)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)

AIC = 2.0*DF8 - 2.0*LLK8
BIC = DF8 *np.log(NSample) - 2.0 * LLK8
print("AIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.195811
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9980
Method:                           MLE   Df Model:                           18
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.8168
Time:                        01:49:37   Log-Likelihood:                -1958.1
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0179      0.087     11.657      0.000       0.847       1.189
x9             0.0073      0

In [ ]:
# Consider Model 1 is y = intercept + x1 + x2+ x3 + x4 + x5 + x6 + x7 + x8 + x10
DriveTrain = df2[['x10']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x1','x2','x3','x4','x5','x6','x7','x8']])
X = stats.add_constant(X, prepend=True)
DF9 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK9 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK9)
DF = DF1 - DF9
pValue = scipy.stats.chi2.sf(Deviance, DF)
print(thisFit.summary())
print("Removed value: x9")
print("Model Log-Likelihood Value =", LLK9)
print("Number of Free Parameters =", DF9)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)

AIC = 2.0*DF9 - 2.0*LLK9
BIC = DF9 *np.log(NSample) - 2.0 * LLK9
print("AIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.195695
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9980
Method:                           MLE   Df Model:                           18
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.8169
Time:                        01:50:07   Log-Likelihood:                -1957.0
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0141      0.087     11.624      0.000       0.843       1.185
x10            0.0070      0

In [ ]:
# Consider Model 1 is y = intercept + x1 + x2+ x3 + x4 + x5 + x6 + x7 + x8 + x9
DriveTrain = df2[['x1']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x9','x2','x3','x4','x5','x6','x7','x8']])
X = stats.add_constant(X, prepend=True)
DF10 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK10 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK10)
DF = DF1 - DF10
pValue = scipy.stats.chi2.sf(Deviance, DF)
print(thisFit.summary())
print("Removed value: x10")
print("Model Log-Likelihood Value =", LLK10)
print("Number of Free Parameters =", DF10)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)

AIC = 2.0*DF10 - 2.0*LLK10
BIC = DF10 *np.log(NSample) - 2.0 * LLK10
print("AIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.811114
         Iterations 7
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9980
Method:                           MLE   Df Model:                           18
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.2411
Time:                        01:50:43   Log-Likelihood:                -8111.1
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5415      0.041     13.191      0.000       0.461       0.622
x1            -0.4959      0.

In [ ]:
# Consider Model 1 is y = intercept + x1 + x4 + x10
DriveTrain = df2[['x1']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x4','x10']])
X = stats.add_constant(X, prepend=True)
DF0 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK0 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK0)
DF = DF1 - DF0
pValue = scipy.stats.chi2.sf(Deviance, DF)
print(thisFit.summary())
print("Removed Features: x2,x3,x5,x6,x7,x8,x9")
print("Model Log-Likelihood Value =", LLK0)
print("Number of Free Parameters =", DF0)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)

AIC = 2.0*DF0 - 2.0*LLK0
BIC = DF0 *np.log(NSample) - 2.0 * LLK0
print("AIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.196190
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9992
Method:                           MLE   Df Model:                            6
Date:                Tue, 09 Mar 2021   Pseudo R-squ.:                  0.8164
Time:                        01:54:17   Log-Likelihood:                -1961.9
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0168      0.087     11.675      0.000       0.846       1.188
x1            -1.1145      0

In [ ]:
# Consider Model 1 is y = intercept +  x4 + x10
DriveTrain = df2[['x4']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x10']])
X = stats.add_constant(X, prepend=True)
DF0 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK0 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK0)
DF = DF1 - DF0
pValue = scipy.stats.chi2.sf(Deviance, DF)
print("Removed Features:x1, x2,x3,x5,x6,x7,x8,x9")
print("Model Log-Likelihood Value =", LLK0)
print("Number of Free Parameters =", DF0)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)

AIC = 2.0*DF0 - 2.0*LLK0
BIC = DF0 *np.log(NSample) - 2.0 * LLK0
print("AIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.223041
         Iterations 10
Removed Features:x1, x2,x3,x5,x6,x7,x8,x9
Model Log-Likelihood Value = -2230.4083138546553
Number of Free Parameters = 6
Deviance (Statistic, DF, Significance) 548.7063482131143 16 1.6859792210470048e-106
AIC: 4472.8166277093105 
BIC: 4516.078669941167


In [ ]:
# Consider Model 1 is y = intercept +  x1 + x10
DriveTrain = df2[['x1']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x10']])
X = stats.add_constant(X, prepend=True)
DF0 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK0 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK0)
DF = DF1 - DF0
pValue = scipy.stats.chi2.sf(Deviance, DF)
print("Removed Features: x2,x3,x4,x5,x6,x7,x8,x9")
print("Model Log-Likelihood Value =", LLK0)
print("Number of Free Parameters =", DF0)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)

AIC = 2.0*DF0 - 2.0*LLK0
BIC = DF0 *np.log(NSample) - 2.0 * LLK0
print("AIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.578737
         Iterations 9
Removed Features: x2,x3,x4,x5,x6,x7,x8,x9
Model Log-Likelihood Value = -5787.3663888876135
Number of Free Parameters = 6
Deviance (Statistic, DF, Significance) 7662.622498279031 16 0.0
AIC: 11586.732777775227 
BIC: 11629.994820007085


In [ ]:
# Consider Model 1 is y = intercept +  x1 + x4
DriveTrain = df2[['x1']]
X = pd.get_dummies(DriveTrain)
X = X.join(df2[['x4']])
X = stats.add_constant(X, prepend=True)
DF0 = numpy.linalg.matrix_rank(X) * (len(y_category) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK0 = logit.loglike(thisParameter.values)

Deviance = 2 * (LLK1 - LLK0)
DF = DF1 - DF0
pValue = scipy.stats.chi2.sf(Deviance, DF)
print("Removed Features: x2,x3,x5,x6,x7,x8,x9,x10")
print("Model Log-Likelihood Value =", LLK0)
print("Number of Free Parameters =", DF0)
print("Deviance (Statistic, DF, Significance)", Deviance, DF, pValue)

AIC = 2.0*DF0 - 2.0*LLK0
BIC = DF0 *np.log(NSample) - 2.0 * LLK0
print("AIC:",AIC,"\nBIC:",BIC)

Optimization terminated successfully.
         Current function value: 0.811988
         Iterations 7
Removed Features: x2,x3,x5,x6,x7,x8,x9,x10
Model Log-Likelihood Value = -8119.8835923191655
Number of Free Parameters = 6
Deviance (Statistic, DF, Significance) 12327.656905142136 16 0.0
AIC: 16251.767184638331 
BIC: 16295.029226870189
